In [36]:
from pathlib import Path
Path("HistoQuant/nuc_seg/raw/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/raw/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/test/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/test/mask").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/results").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train_aug/source").mkdir(parents=True, exist_ok=True)
Path("HistoQuant/nuc_seg/train_aug/mask").mkdir(parents=True, exist_ok=True)

In [37]:
# Extract usefull file from archive
import os
import tarfile
img_archive = tarfile.open("EHRoes/image_data.tar.gz")
for file in img_archive.getmembers():
    if file.isreg():
        file.name = os.path.basename(file.name) 
        if ".gitkeep" in file.name: continue
        elif "images_db.csv" in file.name: continue
        elif "_blend_image.png" in file.name: continue
        elif "_classifier.pkl" in file.name: continue
        elif "_mask_annot.json" in file.name: continue
        elif "_seq_matrix.npy" in file.name: img_archive.extract(file, "HistoQuant/nuc_seg/raw/mask")
        elif "_mask_image.png" in file.name: img_archive.extract(file, "HistoQuant/nuc_seg/raw/mask")
        else: img_archive.extract(file, "HistoQuant/nuc_seg/raw/source")

In [38]:
# Make Tuple of all (source img, mask img, numpy matrix)
import glob
all_raw = glob.glob("HistoQuant/nuc_seg/raw/source/*")
all_mask_img = glob.glob("HistoQuant/nuc_seg/raw/mask/*_mask_image.png")
all_mask_matrix = glob.glob("HistoQuant/nuc_seg/raw/mask/*.npy")
all_data_tuple = []
for i in range(len(all_mask_img)):
    all_data_tuple.append((all_raw[i], all_mask_img[i], all_mask_matrix[i]))

In [39]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(all_data_tuple, train_size=0.70, random_state=42)

In [40]:
import shutil
from PIL import Image, ImageOps

# Extract Images and convert all to same format, same name different folder

for data_tuple in train:
    raw = data_tuple[0]
    mask = data_tuple[1]
    npy_matrix = data_tuple[2]
    filename = raw.split("/")[-1] # Remove full path
    filename = filename.split(".")[0] # Remove extension
    filename = filename + ".png" # Add extension

    with Image.open(raw) as im:
        gray_image = ImageOps.grayscale(im)
        gray_image.save("HistoQuant/nuc_seg/train/source/" + filename, "PNG")
    with Image.open(mask) as im:
        rgb_im = im.convert('RGB')
        gray_image = ImageOps.grayscale(rgb_im)
        gray_image.save("HistoQuant/nuc_seg/train/mask/" + filename, "PNG")
        
    #shutil.copy(npy_matrix, "HistoQuant/nuc_seg/train/mask")

for data_tuple in test:
    raw = data_tuple[0]
    mask = data_tuple[1]
    npy_matrix = data_tuple[2]
    filename = raw.split("/")[-1] # Remove full path
    filename = filename.split(".")[0] # Remove extension
    filename = filename + ".png" # Add extension

    with Image.open(raw) as im:
        gray_image = ImageOps.grayscale(im)
        gray_image.save("HistoQuant/nuc_seg/test/source/" + filename, "PNG")
    with Image.open(mask) as im:
        rgb_im = im.convert('RGB')
        gray_image = ImageOps.grayscale(rgb_im)
        gray_image.save("HistoQuant/nuc_seg/test/mask/" + filename, "PNG")
    # shutil.copy(npy_matrix, "HistoQuant/nuc_seg/test/mask")